# Introduction to Convolutional Neural Network and computer Vision with Tensorflow
Computer vision is the practice of writing algorithms which can discover pattern in visual data. Such as the camera of self driving car recognizing the car in front.

In [ ]:
# !pip install tensorflow

In [ ]:
import zipfile

! wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

In [ ]:
# Unzip the file
zip_ref = zipfile.ZipFile("pizza_steak.zip")
zip_ref.extractall()
zip_ref.close()

## Inspect the data ( become one with it)
A very crucial step at the beginning of any machine learning project is becoming one with the data.
And for a computer vision project .... this usually mean visualize many sample of your data.


In [ ]:
!ls pizza_steak

In [ ]:
!ls pizza_steak/train/

In [ ]:
!ls pizza_steak/train/pizza

In [ ]:
import os
import tensorflow as tf

# walk through pizza_steak directory and list number of files
for dirpath, dirnames,filesnames in os.walk("pizza_steak"):
  print(f"There are {len(dirnames)} directories and {len(filesnames)} images in {dirpath} ")


In [ ]:
# Another way to find out how many images are in a file
num_steak_images_train = len(os.listdir("pizza_steak/train/steak"))
num_pizza_images_train =len(os.listdir("pizza_steak/train/pizza"))
(num_steak_images_train,num_pizza_images_train)

In [ ]:
# Get the classnames programmatically
import pathlib
import numpy as np
data_dir = pathlib.Path("pizza_steak/train")
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))


In [ ]:
# Let's visualize our images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
def view_random_image(target_dir,target_class):
  # Setup the target directory (we'll view images from here)
  target_folder = target_dir + "/" + target_class
  # Get a random image path
  random_image = random.sample(os.listdir(target_folder),1)
  # Read in the image and plot it using matplotlib
  img = mpimg.imread(target_folder + "/" + random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off");
  print(f"Image shape: {img.shape}") # show the shape of the image
  return img



In [ ]:
# View a random image from the training dataset
img = view_random_image(target_dir="pizza_steak/train",target_class="pizza")


In [ ]:
tensor_image = tf.constant(img/255.0)
tf.shape(tensor_image),tensor_image

In [ ]:
"""
Load our images
preprocess our images
Build a CNN to find pattterns in our images
Compile our CNN
fit the CNN to our training data


"""
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Set the seed
tf.random.set_seed(42)
# Preprocess data (get all of the pixel values between 0 & 1,also called normalize/scaling)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
# Setup paths to our data directories
train_dir = "/content/pizza_steak/train"
test_dir = "/content/pizza_steak/test"
# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               batch_size=32,
                                               target_size=(224,224),
                                               class_mode="binary",
                                               seed=42)
valid_data = valid_datagen.flow_from_directory(directory=test_dir,
                                               batch_size=32,
                                               target_size=(224,224),
                                               class_mode="binary",
                                               seed=42)
# Build a CNN model
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=10,
                           kernel_size=3,
                           activation="relu",
                           input_shape=(224,224,3)),

    tf.keras.layers.Conv2D(10,3,activation="relu"),

    tf.keras.layers.MaxPool2D(pool_size=2,
                              padding="valid"),
    tf.keras.layers.Conv2D(10,3,activation="relu"),
    tf.keras.layers.Conv2D(10,3,activation="relu"),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(280,activation="relu"),
    # tf.keras.layers.Dense(140,activation="relu"),
    # tf.keras.layers.Dense(70,activation="relu"),
    # tf.keras.layers.Dense(30,activation="relu"),
    # tf.keras.layers.Dense(5,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
    ])
# Compile our CNN
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(),
                metrics=["accuracy"])
# Fit the model
history_model_1 = model_1.fit(train_data,
                              epochs=5,
                              steps_per_epoch=len(train_data),
                              validation_data= valid_data,
                              validation_steps=len(valid_data))


In [ ]:
model_1.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model_1,show_shapes=True,show_layer_names=True,expand_nested=True)

In [ ]:
plt.plot(range(len(history_model_1.history["accuracy"])),history_model_1.history["accuracy"],label="accuracy", color="green")
plt.plot(range(len(history_model_1.history["loss"])),history_model_1.history["loss"],label="loss",color="red")
plt.title("loss and accuracy of food classification CNN model over 25 epochs")
plt.xlabel("epochs")
plt.ylabel("loss and accuracy")
plt.legend();

In [ ]:
# Set random seed
tf.random.set_seed(42)

# Create a model to replicate
model_2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224,224,3)),
    tf.keras.layers.Dense(4,activation="relu"),
    tf.keras.layers.Dense(4,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")

])

# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"]
                )
# Fit the model
history_2 = model_2.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data)
                        )


In [ ]:
# Set random seed
tf.random.set_seed(42)
# create the model
model_3 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224,224,3)),
    tf.keras.layers.Dense(100,activation="relu"),
    tf.keras.layers.Dense(100,activation="relu"),
    tf.keras.layers.Dense(100,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
# Fit the model
history_3 = model_3.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

In [ ]:
model_2.summary(),model_3.summary()

## Binary classification: Let's break it down
1. Become on with the data(visualize,visualize,visualize)
2. Preprocess the data(prepared it for our model, the main step her was scaling/normalizing)
3. Create a model(start with abaseline)
4. Fit the model
5. Evaluate the model
6. adjust different parameters and improve the model(try to beat our baseline)
7. Repeat until satisfied (experiment,experiment,experiment)

### Become one with data

In [ ]:
# Visualize data
plt.figure()
plt.subplot(1,2,1)
steak_img = view_random_image("pizza_steak/train/","steak")
plt.subplot(1,2,2)
pizza_img = view_random_image("pizza_steak/train/","pizza")



### Preproccess the data( prepare it for a model)


In [ ]:
# Define directory dataset paths
train_dir = "pizza_steak/train"
test_dir = "pizza_steak/test"

In [ ]:
# tern our data into Batchs
!nvidia-smi

In [ ]:
# Create train and test data generators and rescale the data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
# Load in our image data from directories and trun them into batches
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size=(224,224),
                                               class_mode="binary",
                                               batch_size=32)
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                               target_size=(224,224),
                                               class_mode="binary",
                                               batch_size=32)

In [ ]:
# Get a sample of a train data batch
print(len(train_data))

In [ ]:
# Make the creating of our model a little easier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D,Activation
from tensorflow.keras import Sequential



In [ ]:
# Create the model
model_4 = Sequential([
    Conv2D(filters=10,
           kernel_size=3,
           strides=1,
           padding="valid",
           activation="relu",
           input_shape=(224,224,3)), # input layer

    Conv2D(10,3, activation="relu"),
    Conv2D(10,3, activation="relu"),
    Flatten(),
    Dense(1,activation="sigmoid") # output layer
])

In [ ]:
# compile the model
model_4.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])
model_4.summary()

In [ ]:
len(train_data),len(test_data)

In [ ]:
# fit the model
history_model_4 = model_4.fit(train_data,
            epochs=5,
            steps_per_epoch=len(train_data),
            validation_data = test_data,
            validation_steps=len(test_data)
            )

In [ ]:
model_1.evaluate(test_data),model_4.evaluate(test_data)

In [ ]:
# Let's plot the training curves
import pandas as pd
pd.DataFrame(history_model_4.history).plot(figsize=(7,5))

In [ ]:
# plot the validation and training curves separatedly
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics
  """
  loss = history.history["loss"]
  val_loss = history.history["val_loss"]
  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]
  epochs = range(len(history.history["loss"]))
  # Plot loss
  plt.figure();
  plt.plot(epochs,loss,label="training_loss",color="red")
  plt.plot(epochs,val_loss,label="validation_loss")
  plt.title("loss")
  plt.xlabel("epoch")
  plt.legend()

  #plot accuracy
  plt.figure();
  plt.plot(epochs,accuracy,label="training_accuracy",color="green")
  plt.plot(epochs,val_accuracy,label="validation_accuracy")
  plt.title("validation")
  plt.xlabel("epochs")
  plt.legend()



In [ ]:
plot_loss_curves(history=history_model_4)

 #### Adjust te model parameters
 Fitting a machine learning model comes in 3 steps:

 0. Create a baseline
 1. Beat the beseline by overfitting a larger model
 2. Reduce overfitting

 Ways to induce overfitting:
 * Increase the number of conv layers
 * Increase the number of conv filters
 * add another dense layer to the ouput of our flattened layer

 Reduce overfitting:
 * add data augmentation
 * add regularization layers(such as MaxPool23)
 * add more data...


In [ ]:
# Create the model (this is going to be our new baseline)
model_5 = Sequential([
    Conv2D(10,3,activation="relu",input_shape=(224,224,3)),
    MaxPool2D(pool_size=2),
    Conv2D(10,3,activation="relu"),
    MaxPool2D(),
    Conv2D(10,3,activation="relu"),
    MaxPool2D(),
    Flatten(),
    Dense(1,activation="sigmoid")

])

In [ ]:
# Compile the model
model_5.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])
# Fit the model
history_model_5 = model_5.fit(train_data,
                              epochs=5,
                              steps_per_epoch=len(train_data),
                              validation_data=test_data,
                              validation_steps=len(test_data)
                              )

In [ ]:
# Get a summary our model with max pooling
model_5.summary()

In [ ]:
model_4.summary()

In [ ]:
# plot training curves of model_5
plot_loss_curves(history_model_5)

In [ ]:
# compare model_4 and model_5 so as to know the power of maxpool
comparision_data = dict()
comparision_data["accuracy"] = history_model_5.history["accuracy"]
comparision_data["val_accuracy"] = history_model_4.history["accuracy"]
comparision_data["loss"] = history_model_5.history["loss"]
comparision_data["val_loss"] = history_model_4.history["loss"]

# plot comparition curve
# plot_loss_curves(comparision_data)


### Opening our bag of tricks and finding data augmentation

In [ ]:
# Create ImageDataGenerator training instance with data augmentation
train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range=0.2,
                                             shear_range=0.2,
                                             zoom_range=0.2,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             horizontal_flip=True
                                             )
# Create ImageDataGenerator without data augmentation
train_datagen =  ImageDataGenerator(rescale=1/255.)

# Create ImageDataGenerator without data augmentation for the test dataset
test_datagen = ImageDataGenerator(rescale=1/255.)

> **Question:** what is data augmentation?
Data augmentation is the process of altering our training data leading it to have more diversity and in turn allowing our models to learn more genralizable ( hopefully) patterns.
Altering might mean adjusting the rotation of an image, flipping it, cropping it or something similar.

In [ ]:
# Import data and augment it from training directory
print("Augmented training data:")
train_dir = "/content/pizza_steak/train"
train_data_augmented = train_datagen_augmented.flow_from_directory(train_dir,
                                                                   target_size=(224,224),
                                                                   batch_size=32,
                                                                   class_mode="binary",
                                                                   shuffle=False)
# Create non-augmented train data batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode="binary",
                                               shuffle=False)
# Create non-augmented test data batchs
test_data_nonaugmented = test_datagen.flow_from_directory(test_dir,
                                                          target_size=(224,224),
                                                          batch_size=32,
                                                          class_mode="binary"
                                                         )



In [ ]:
# Get sample augmented data batches
images,labels = next(train_data)
augmented_images,augmented_labels = next(train_data_augmented)
images[0]

In [ ]:
# Show original image and augmented image
import random
random_number = random.randint(0,31)
print(f"showing image number: {random_number}")
plt.imshow(images[random_number])
plt.title("Original image")
plt.axis("off")
plt.figure();
plt.imshow(augmented_images[random_number])
plt.title("Augmented image")
plt.axis("off")

In [ ]:
# Create the model
model_6 = Sequential([
    Conv2D(10,3,activation="relu"),
    MaxPool2D(pool_size=2),
    Conv2D(10,3,activation="relu"),
    MaxPool2D(),
    Conv2D(10,3,activation="relu"),
    MaxPool2D(),
    Flatten(),
    Dense(1,activation="sigmoid")

])

# Compile the model
model_6.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])
# Fit the model
history_6 = model_6.fit(train_data_augmented,
                        epochs=5,
                        steps_per_epoch=len(train_data_augmented),
                        validation_data=test_data,
                        validation_steps=len(test_data))

In [ ]:
# Show model_6 training curves
plot_loss_curves(history_6)

## Let's shuffle our augmented training data and train another model(the same as before) on it and see what happens.

In [ ]:
# Import data and augment it and shuffle from training directory
train_data_augmented_shuffled = train_datagen_augmented.flow_from_directory(train_dir,
                                                                target_size=(224,224),
                                                                batch_size=32,
                                                                class_mode="binary",
                                                                shuffle=True # shuffle data now
                                                                )

In [ ]:
train_data_augmented_shuffled

In [ ]:
# Create the model
model_7 = Sequential([
    Conv2D(10,3,activation="relu"),
    MaxPool2D(),
    Conv2D(10,3,activation="relu"),
    MaxPool2D(),
    Conv2D(10,3,activation="relu"),
    MaxPool2D(),
    Flatten(),
    Dense(1,activation="sigmoid")
])

# Compile the model
model_7.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])
# Fit the model
history_7 = model_7.fit(train_data_augmented_shuffled,
            epochs=5,
            steps_per_epoch=len(train_data_augmented_shuffled),
            validation_data=test_data,
            validation_steps=len(test_data)
            )

In [ ]:
# plot loss curves of model_7
plot_loss_curves(history_7)

### 7.Repeat untill satisfied
Since we already beaten our baseline, there are a few things we could try to contune to improve our model:
* add more layers `Conv2D`/`MaxPool2D`
* Increase the number of filters in each convolutional layers( e.g from 10 to 32 or 64)
* Train for longer (more epochs)
* Find an ideal learning rate rearange tensor value
* Get more data (give the model more opportunities to learn)
* Use **Transfer Learnig** to leverage what another image model has learn and adjust it for our own use case

## Making a prediction with our trained model on our own custom data

In [ ]:
# Classes we're working with
print(class_names)

In [ ]:
!wget https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/images/03-steak.jpeg


In [ ]:
!ls -la

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf



In [ ]:
# print(os.path.exists("/content/03-steak.jpeg"))
# from PIL import  Image
# Image.open("/content/03-steak (1).jpeg")
steak = mpimg.imread('/content/03-steak (1).jpeg')
plt.imshow(steak)

In [ ]:
# resize the image to match what model_7 expext
steak_resized = tf.image.resize(steak,[224,224])
steak_resized = tf.cast(steak_resized,dtype=tf.int32)

# show resized image
plt.imshow(steak_resized)
plt.axis(False)


In [ ]:
# get prediction of resized image
model_7.predict(steak_resized)

In [ ]:
# add batch size to the image before feeding to the model
steak_resized = tf.expand_dims(steak_resized,axis=0)
model_7.predict(steak_resized)

In [ ]:
# test model with Pizza image
pizza = mpimg.imread("/content/pizza.jfif")
plt.imshow(pizza)

In [ ]:
# Resize and add batch size to the image so as to pass model_7 requirement exam
resized_pizza = tf.image.resize(pizza,[224,224])
# show resized pizza image
plt.imshow(resized_pizza)
# add extra dimention for batch size
resized_batched_pizza = tf.expand_dims(resized_pizza,axis=0)


In [ ]:
!ls drive/MyDrive/'Colab Noteooks'